In [1]:
!pip install pillow

In [2]:
from PIL import Image
import numpy as np

In [3]:
def encode_message(img_path, message, output_path):
    img = Image.open(img_path).convert('RGB')
    data = np.array(img)

    flat_data = data.flatten()

    # Convert message to binary and add a delimiter
    binary_message = ''.join(format(ord(char), '08b') for char in message) + '1111111111111110'

    if len(binary_message) > len(flat_data):
        raise ValueError("Message is too long to hide in this image.")

    # Modify LSB of each byte with message bits
    for i in range(len(binary_message)):
        flat_data[i] = (flat_data[i] & 0xFE) | int(binary_message[i])  # Safe bit masking

    # Reshape and save the image
    new_data = flat_data.reshape(data.shape)
    encoded_img = Image.fromarray(new_data.astype('uint8'), 'RGB')
    encoded_img.save(output_path)
    print("✅ Message encoded and saved as:", output_path)

In [4]:
def decode_message(img_path):
    img = Image.open(img_path)
    img = img.convert('RGB')
    data = np.array(img).flatten()

    binary_message = ''
    for value in data:
        binary_message += str(value & 1)

    # Split into 8-bit chunks
    chars = [binary_message[i:i+8] for i in range(0, len(binary_message), 8)]

    message = ''
    for c in chars:
        if c == '11111110':  # EOF marker
            break
        message += chr(int(c, 2))
    return message

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
# Example usage
encode_message("sample.jpg", "Hello from steganography!", "encoded_image.png")

In [ ]:
hidden_message = decode_message("encoded_image.png")
print("Hidden message:", hidden_message)